In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tasks import NER
from utils_ner import TokenClassificationDataset
import torch
import  nltk

In [7]:
# load phobert-ner đã được fine-tuned
phobert_ner = AutoModelForTokenClassification.from_pretrained("huyenbui117/Covid19_phoNER")
# load tokenizer của phobert
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
labels = NER().get_labels(None)
labels

['O',
 'B-ORGANIZATION',
 'O-ORGANIZATION',
 'I-ORGANIZATION',
 'B-LOCATION',
 'I-LOCATION',
 'B-SYMPTOM_AND_DISEASE',
 'I-SYMPTOM_AND_DISEASE',
 'B-DATE',
 'I-DATE',
 'B-PATIENT_ID',
 'I-PATIENT_ID',
 'B-AGE',
 'I-AGE',
 'B-NAME',
 'I-NAME',
 'B-JOB',
 'I-JOB',
 'B-TRANSPORTATION',
 'I-TRANSPORTATION',
 'B-GENDER']

In [11]:
sentences = ["Thành phố Nam Định"]

In [15]:
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    sequence = " ".join(tokens)
    input_ids = torch.tensor([tokenizer.encode(sentence, "utf-8")])

    print(tokens)
    outputs = phobert_ner(input_ids).logits
    predictions = torch.argmax(outputs, dim=2)
    print(predictions)
    for i in [(token, labels[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]:
        print(i)
    print('---------------------------------------------')

['Thành', 'phố', 'Nam', 'Định']
tensor([[0, 0, 5, 5, 5, 0, 0, 0, 0, 0, 0]])
('Thành', 'O')
('phố', 'O')
('Nam', 'I-LOCATION')
('Định', 'I-LOCATION')
---------------------------------------------


In [ ]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP(address="http://127.0.0.1", port=9000)
word_segmented_text = annotator.tokenize(sentence for sentence in sentences)

KeyboardInterrupt: 